In [ ]:
import os
from os.path import join
import itertools

from keras.layers import Input, Conv2D, Dropout, MaxPool2D, GlobalAveragePooling2D, GlobalMaxPooling2D, LeakyReLU
from keras.layers import merge, Activation
from keras.models import Model
from keras.optimizers import Adadelta
from keras.utils import to_categorical

from common.paths import PATH_PROJECT 

In [ ]:
class DarknetBlock:
    def __init__(self, filters1, filters3, strides=(1, 1)):
        self._filters1 = filters1
        self._filters3 = filters3
        self._strides = strides
        
    def __call__(self, input_layer, *args, **kwargs):
        x = input_layer
        for filters, kernel, strides in zip([self._filters1, self._filters3],
                                            [(1, 1), (3, 3)],
                                            [(1, 1), self._strides]):
            x = Conv2D(filters, kernel, padding='same', strides=strides)(x)
            x = LeakyReLU()(x)
        return x


def build_model(class_count, is_train=False):
    input_tensor = Input(shape=(None, 1, None, None))
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    for filters in [16, 32, 64]:
        x = DarknetBlock(filters, filters * 2)(x)
        x = DarknetBlock(filters, filters * 2)(x)
        x = MaxPool2D(pool_size=(2, 2), padding='same')(x)
        if is_train:
            x = Dropout(0.1)(x)
    
    x = Conv2D(class_count, (3, 3), padding='same')(x)
    max_output = GlobalMaxPooling2D()(x)
    average_output = GlobalAveragePooling2D()(x)
    x = merge(inputs=[max_output, average_output], mode='sum', concat_axis=1)
    x = Activation('softmax')(x)
    return Model(inputs=[input_tensor], outputs=[x])

In [ ]:
CLASS_COUNT = 6
BATCH_SIZE = 32

In [ ]:
def read_label_map(path):
    return {}


def is_png_image(path):
    return os.path.isfile(path) and os.path.splitext(path) == '.png'


def get_image_paths(root):
    def _get_image_paths():
        for path, dirs, files in os.walk(root):
            file_paths = map(lambda filename: join(path, filename), files)
            images = filter(is_png_image, file_paths)
            yield images
    return itertools.chain.from_iterable(_get_image_paths())


PATH_DATA = join(PATH_PROJECT, 'contest1', 'data')

label_map = read_label_map(join(PATH_DATA, 'train_labels.csv'))
images_paths_train = list(get_image_paths(join(PATH_DATA, 'train')))
images_paths_test = list(get_image_paths(join(PATH_DATA, 'test')))
labels_train = (for image_path in images_paths_train)

In [ ]:
TRAIN_SIZE, TEST_SIZE = len(images_train), len(images_test)

In [ ]:
model = build_model(CLASS_COUNT, True)
model.compile(optimizer=Adadelta, loss='categorical_crossentropy')
model.summary()

In [ ]:
history = model.fit_generator(generator=data_generator,
                              steps_per_epoch=)